# Code playground
This notebook is a small sandbox where half baked ideas can live. Code snippets that were developed but might be useful saterr can go here.

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchtext
import tqdm
from torchnlp.metrics import get_moses_multi_bleu
from torchtext.data import Field, BucketIterator
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

import tensorflow as tf
import tensorflow_datasets as tfds
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

import linecache
import sys
import os
import re
import random
import time

In [4]:
torch.tensor([1,4,3,6,2]).argsort()

tensor([0, 4, 2, 1, 3])

## Self-made beam search code
This code runs on paralelised batches and beams, it's fast, but doesn't have the appropriate stopping conditions.

In [ ]:
def beam_search_decode_batch_ids(encoder_input, beam_size=3, max_seq_length=50):
    batch_len = encoder_input.shape[1]
    sos_id = TGT_TEXT.vocab.stoi["<sos>"]
    decoder_input = torch.zeros((1, beam_size * batch_len), dtype=torch.long, device=device).fill_(sos_id)
    log_probs = torch.zeros((beam_size * batch_len,1))
    print("log_probs:", log_probs)
    vocab_size = len(TGT_TEXT.vocab.itos)
    
    encoder_input = encoder_input.view(-1,1).repeat(1,beam_size).view(-1,beam_size * batch_len) # this tiles the input to the beam size * batch size
    print("tiled input:", encoder_input)
    
    for i in range(max_seq_length):
        output = model(encoder_input, decoder_input)
        print(output.shape)
        last_pred = output[-1].softmax(1)
        print("last_pred shape:", last_pred.shape)
        log_predictions = last_pred.log()
        seq_log_probs = log_predictions + log_probs.repeat(1,vocab_size)
        print("seq_log_probs:", seq_log_probs)
        
        seq_log_probs_positions = seq_log_probs.view(batch_len,-1).argsort(1)[:,-beam_size:]
        print("seq_log_probs_positions:", seq_log_probs_positions)
        next_ids = seq_log_probs_positions.reshape(1, batch_len*beam_size) % vocab_size
        print("next_ids:", next_ids)
        log_probs = seq_log_probs.view(batch_len,-1).gather(1,seq_log_probs_positions).view(beam_size * batch_len,1)
        print("log_probs:",log_probs)
        
        for batch_idx in range(seq_log_probs_positions.shape[0]):
            for seq_choice_idx in range(seq_log_probs_positions.shape[1]):
                seq_choice = seq_log_probs_positions[batch_idx,seq_choice_idx] // vocab_size
                decoder_input[:,batch_idx*batch_len + seq_choice_idx] = decoder_input[:,batch_idx*batch_len + seq_choice]
        
        decoder_input = torch.cat((decoder_input, next_ids))
    return decoder_input
        
#         last_pred = output[-1].argsort(dim=1)[:,:beam_size]

sent1 = ["<sos>"] + SRC_TEXT.preprocess("create array") + ["<eos>"] + ["<pad>"]
sent2 = ["<sos>"] + SRC_TEXT.preprocess("if exists then") + ["<eos>"]
src_ids = SRC_TEXT.numericalize([sent1, sent2], device=device)
print("input ids:", src_ids)
beam_search_decode_batch_ids(src_ids, max_seq_length=5)